In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

#from tensorflow.keras import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
#from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
from keras import Sequential
from keras.optimizers import Adam
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [3]:
import cv2
from PIL import Image

In [4]:
df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/train.csv")
df_test = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/test.csv")

In [5]:
df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [57]:
# True image size
IMG_WIDTH = 200
IMG_HEIGHT = 200

def preprocess_image(image, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    
    :param img: A NumPy Array that will be cropped
    :param sigmaX: Value used for add GaussianBlur to the image
    
    :return: A NumPy array containing the preprocessed image
    """
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image

In [58]:
y = df["Class"]

In [59]:
from matplotlib import pyplot as plt
%matplotlib inline

In [60]:
#img = cv2.imread(r'E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/image7042.jpg')
#cv2.imshow('image',img)
#cv2.waitKey(0)

In [61]:
# MAKING SURE ALL IMAGES ARE OF SAME SIZE

#diff_shape = set()
PATH = "E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/"
#for i in range(df.shape[0]):
#    temp = PATH + str(df["Image"][i])
#    img = cv2.imread(temp)
#    diff_shape.add(img.shape)

In [62]:
BATCH_SIZE = 4
# resizing the image to the size of efficientnet

# Add Image augmentation to our generator
train_datagen = ImageDataGenerator(rotation_range=90,
                                 brightness_range=[0.2,1.0],
                                   zoom_range=[0.5,1.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.1,
                                   preprocessing_function=preprocess_image, 
                                   rescale=1 / 128.)
train_generator = train_datagen.flow_from_dataframe(df, 
                                                    x_col='Image', 
                                                    y_col='Class',
                                                    directory = PATH,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical', 
                                                    subset='training')
val_generator = train_datagen.flow_from_dataframe(df, 
                                                  x_col='Image', 
                                                  y_col='Class',
                                                  directory = PATH,
                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  subset='validation')

Found 5385 validated image filenames belonging to 4 classes.
Found 598 validated image filenames belonging to 4 classes.


In [63]:
## seems like efficient net is too large
# can try to freeze some layers


#from keras_efficientnets import EfficientNetB3
#effnet = EfficientNetB3(input_shape=(300,300,3),
#                        weights='imagenet',
#                        include_top=False)

In [64]:
#effnet.summary()

In [65]:
from keras.regularizers import l2
l2 = l2(0.01)

In [66]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(r'E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset\Models/mynet_v3.h5', monitor='categorical_crossentropy', mode='min', verbose=1, save_best_only=True)

In [67]:
df_test.head(10)

,Image,Class
0,image6245.jpg,Attire
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Food
5,image848.jpg,Food
6,image1232.jpg,misc
7,image8386.jpg,Food
8,image9828.jpg,misc
9,image9033.jpg,Food


In [68]:
test_path = "E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset\Test Images/"

In [69]:
df_test['Class'] = np.zeros(df_test.shape[0])
df_test["Class"] = df_test["Class"].apply(str)
df_test["Class"] = df_test["Class"].astype("category")
# For preprocessing test images
test_generator = ImageDataGenerator(preprocessing_function=preprocess_image,rescale=1 / 256.).flow_from_dataframe(df_test,
                                                                          x_col = "Image",
                                                                          #y_col = "Class",
                                                                          directory=test_path,
                                                                          target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                                          batch_size=1,
                                                                          class_mode=None,
                                                                                       
                                                                          shuffle=False)
filename = test_generator.filenames
number_test = len(filename)

train_gen_pred = train_datagen.flow_from_dataframe(df, 
                                                    x_col='Image', 
                                                    directory = PATH,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=1,
                                                    class_mode=None, 
                                                    shuffle = False)

Found 3219 validated image filenames.
Found 5983 validated image filenames.


In [38]:
#from keras.models import load_model,model_from_json

In [39]:
#from keras_efficientnets import EfficientNetB4
#effnet = EfficientNetB5(input_shape=(456,456,3),
#                        #weights='imagenet',
#                       include_top=False)

In [40]:
#for i in effnet.layers:
#    i.trianable = False


In [56]:
from keras.models import load_model
model = load_model('E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset\Models/mynet_v3.h5')

In [70]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 200, 200, 32)      4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 32)      128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 198, 198, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 196, 196, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 196, 196, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 64)       

In [71]:
predict = model.predict_generator(test_generator,number_test)

In [86]:
def convert_pred(pred):
    output = []
    output1 = []
    for i in pred:
        temp = max(i)
        for c,j in enumerate(i):
            if j == temp:
                output1.append(c)
    for i in output1:
        if i==2:
            output.append("Attire")
        if i==0:
            output.append("Food")
        if i==3:
            output.append("Decorationandsignage")
        if i==1:
            output.append("misc")
    return output1,output

In [87]:
evaluation,submission, = convert_pred(predict)

In [88]:
submission

['Food',
 'Attire',
 'Attire',
 'misc',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Attire',
 'misc',
 'Attire',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Decorationandsignage',
 'Food',
 'Attire',
 'Attire',
 'misc',
 'Food',
 'Attire',
 'Attire',
 'Attire',
 'Decorationandsignage',
 'misc',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Decorationandsignage',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Decorationandsignage',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Decorationandsignage',
 'Attire',
 'Food',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Food',
 'Food',
 'Attire',
 'Attire',
 'Attire',
 'Attire',
 'Food',
 'Attire',
 'Attire',


In [89]:
evaluation

[0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 3,
 0,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 3,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 3,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 3,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 3,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 3,
 2,
 0,
 2,
 0,
 2,
 2,
 1,


In [90]:
df_test = df_test.drop("Class",axis =1)
df_test["Class"] = submission

In [91]:
df_test.to_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/submission.csv",index = False)

In [92]:

for i in range(df_test.shape[0]):
    temp = test_path + str(df_test["Image"][i])
    print(df_test["Class"][i])
    img = cv2.imread(temp)
    cv2.imshow('image',img)
    cv2.waitKey(0)


Food
Attire
Attire
misc
Attire
Attire
Attire
Attire
Attire
Food
Attire
Attire
Attire
Attire
Attire
Attire
Attire
Attire


KeyboardInterrupt: 